In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout

In [2]:
# Google drive mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Setup the train and test directories
train_dir = "drive/MyDrive/Colab Notebooks/dataset/train/"
test_dir = "drive/MyDrive/Colab Notebooks/dataset/test/"

In [4]:
# Create ImageDataGenerator training instance with data augmentation
train_datagen_augmented = ImageDataGenerator(rescale=1/255.,
                                             rotation_range=20, # rotate the image slightly between 0 and 20 degrees
                                             shear_range=0.2, # shear the image
                                             zoom_range=0.2, # zoom into the image
                                             width_shift_range=0.2, # shift the image width ways
                                             height_shift_range=0.2, # shift the image height ways
                                             horizontal_flip=True) # flip the image on the horizontal axis

In [5]:
# Import data and augment it from directories
train_data_augmented_shuffled = train_datagen_augmented.flow_from_directory(train_dir,
                                                                            target_size=(224, 224),
                                                                            batch_size=32,
                                                                            class_mode='categorical',
                                                                            shuffle=True) # Shuffle data (default)

Found 4500 images belonging to 3 classes.


In [6]:
# Create ImageDataGenerator test instance without data augmentation
test_datagen = ImageDataGenerator(rescale=1/255.)

In [7]:
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=(224, 224),
                                             batch_size=32,
                                             class_mode='categorical')

Found 900 images belonging to 3 classes.


In [8]:
model = tf.keras.models.Sequential([Conv2D(filters=32, 
                                           kernel_size=3,
                                           activation="relu", 
                                           input_shape=(224, 224, 3)),
                                    Conv2D(64, 3, activation="relu"),
                                    MaxPool2D(2),
                                    Conv2D(128, 3, activation="relu"),
                                    MaxPool2D(2),
                                    Conv2D(256, 3, activation="relu"),
                                    MaxPool2D(2),
                                    Conv2D(512, 3, activation="relu"),
                                    MaxPool2D(2),
                                    Flatten(),
                                    Dropout(0.7),
                                    Dense(2048, activation = 'relu'),
                                    Dense(3, activation="softmax")])

In [9]:
# Compile the model
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [10]:
# Check out the layers in our model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 128)      0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 52, 52, 256)       2

In [ ]:
# Fit the model
history = model.fit(train_data_augmented_shuffled,
                    epochs=20,
                    steps_per_epoch=len(train_data_augmented_shuffled),
                    validation_data=test_data,
                    validation_steps=len(test_data))

Epoch 1/20
141/141 [==============================] - 2378s 17s/step - loss: 1.1330 - accuracy: 0.3871 - val_loss: 0.9762 - val_accuracy: 0.5511
Epoch 2/20
141/141 [==============================] - 2356s 17s/step - loss: 0.9192 - accuracy: 0.5889 - val_loss: 0.7128 - val_accuracy: 0.7156
Epoch 3/20
 27/141 [====>.........................] - ETA: 35:01 - loss: 0.7905 - accuracy: 0.6678

In [ ]:
def plot_loss_curves(history):
    """
    Returns separate loss curves for training and validation metrics.
    """ 
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    epochs = range(len(history.history['loss']))

    # Plot loss
    plt.plot(epochs, loss, label='training_loss')
    plt.plot(epochs, val_loss, label='val_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot accuracy
    plt.figure()
    plt.plot(epochs, accuracy, label='training_accuracy')
    plt.plot(epochs, val_accuracy, label='val_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend()

In [ ]:
# Check out the loss curves of model
plot_loss_curves(history)

In [ ]:
# View our example image
!wget https://media-cdn.tripadvisor.com/media/photo-s/1c/d0/ba/1a/veggie-pizza.jpg
pizza = mpimg.imread("veggie-pizza.jpg")
plt.imshow(pizza)
plt.show()

In [ ]:
# Check the shape of our image
pizza.shape

In [ ]:
# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224):
    """
    Reads an image from filename, turns it into a tensor
    and reshapes it to (img_shape, img_shape, colour_channel).
    """
    # Read in target file (an image)
    img = tf.io.read_file(filename)

    # Decode the read file into a tensor & ensure 3 colour channels 
    # (our model is trained on images with 3 colour channels
    # but sometimes images have 4 colour channels)
    img = tf.image.decode_image(img, channels=3)

    # Resize the image (to the same size our model was trained on)
    img = tf.image.resize(img, size = [img_shape, img_shape])

    # Rescale the image (get all values between 0 and 1)
    img = img/255.
    return img

In [ ]:
# Load in and preprocess our custom image
pizza = load_and_prep_image("veggie-pizza.jpg")

In [ ]:
pizza

In [ ]:
# Although our image is in the same shape as the images our model
# has been trained on, we're still missing a dimension.
# The batch size becomes the first dimension.
# So in reality, our model was trained on data
# in the shape of (batch_size, 224, 224, 3).
# We can fix this by adding an extra to our custom
# image tensor using tf.expand_dims.

# Add an extra axis
print(f"Shape before new dimension: {pizza.shape}")
pizza = tf.expand_dims(pizza, axis=0) # add an extra dimension at axis 0
print(f"Shape after new dimension: {pizza.shape}")

In [ ]:
pizza

In [ ]:
# Make a prediction on custom image tensor
prediction = model.predict(pizza)

In [ ]:
print(prediction)

In [ ]:
# View our example image
!wget https://www.modular.it/wp-content/uploads/2018/03/risotto-alla-parmigiana.jpeg
risotto = mpimg.imread("risotto-alla-parmigiana.jpeg")
plt.imshow(risotto)
plt.show()

In [ ]:
# Load in and preprocess our custom image
risotto = load_and_prep_image("risotto-alla-parmigiana.jpeg")

In [ ]:
risotto

In [ ]:
# Add an extra axis
print(f"Shape before new dimension: {risotto.shape}")
risotto = tf.expand_dims(risotto, axis=0) # add an extra dimension at axis 0
print(f"Shape after new dimension: {risotto.shape}")

In [ ]:
# Make a prediction on custom image tensor
prediction = model.predict(risotto)

In [ ]:
print(prediction)

In [ ]:
class_names = ["pizza", "risotto"]

In [ ]:
def pred_and_plot(model, filename, class_names):
      """
      Imports an image located at filename, makes a prediction on it with
      a trained model and plots the image with the predicted class as the title.
      """
      # Import the target image and preprocess it
      img = load_and_prep_image(filename)

      # Make a prediction
      pred = model.predict(tf.expand_dims(img, axis=0))

      # Get the predicted class
      pred_class = class_names[int(tf.round(pred)[0][0])]

      # Plot the image and predicted class
      plt.imshow(img)
      plt.title(f"Prediction: {pred_class}")
      plt.show()

In [ ]:
!wget https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/delish-191112-risotto-rice-0151-landscape-pf-1574723947.jpg
food = mpimg.imread("delish-191112-risotto-rice-0151-landscape-pf-1574723947.jpg")

In [ ]:
# Test our model on a custom image
pred_and_plot(model, "delish-191112-risotto-rice-0151-landscape-pf-1574723947.jpg", class_names)